<a href="https://colab.research.google.com/github/Fameless4ellL/GoogleCollabML/blob/main/ML_Project_ZD(w62817).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/content/COMP_07.txt', '\t')
print(data)

     Size   Color  Feel Temperature  Attitude
0     big  yellow  hard         low  positive
1     big  yellow  soft        high  negative
2  medium  yellow  soft        high  positive
3  medium    blue  hard        high  negative
4  medium    blue  soft         low  positive
5  medium    blue  soft         low  positive


In [ ]:
#Function to calculate final Entropy 
def entropy(probs):  
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

#Function to calculate Probabilities of positive and negative examples 
def entropy_of_list(a_list):
    from collections import Counter
    cnt = Counter(x for x in a_list) #Count the positive and negative ex
    num_instances = len(a_list)
    #Calculate the probabilities that we required for our entropy formula 
    probs = [x / num_instances for x in cnt.values()]
    #print(cnt, end=' = ')
    #print(str(num_instances))
    print('-'+str(probs)+' * log2('+str(probs)+')', end=' = ')
    import math 
    print('sum('+str([-prob*math.log(prob, 2) for prob in probs])+')', end=' = ')
    print(sum([-prob*math.log(prob, 2) for prob in probs]))
    #Calling entropy function for final entropy 
    return entropy(probs)

total_entropy = entropy_of_list(data['Attitude'])
print("\n Total Entropy of COMP_07 Data Set:",total_entropy)

-[0.6666666666666666, 0.3333333333333333] * log2([0.6666666666666666, 0.3333333333333333]) = sum([0.38997500048077083, 0.5283208335737187]) = 0.9182958340544896

 Total Entropy of COMP_07 Data Set: 0.9182958340544896


In [ ]:
#Defining Information Gain Function 
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
  print("Information Gain Calculation of ",split_attribute_name)
  print("target_attribute_name",target_attribute_name)
  
  #Grouping features of Current Attribute
  df_split = df.groupby(split_attribute_name)
  for name,group in df_split:
        print("Name: ",name)
        print("Group: ",group)
  nobs = len(df.index) * 1.0
  print("NOBS",nobs)

  #Calculating Entropy of the Attribute and probability part of formula 
  df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
  print("df_agg_ent",df_agg_ent)
  df_agg_ent.columns = ['Entropy', 'Prob1']

  # Calculate Information Gain
  new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['Prob1'] )
  old_entropy = entropy_of_list(df[target_attribute_name])
  print('Entropy of COMP_07 Data('+str(old_entropy)+') - '+str(new_entropy))
  print('Info-gain is :'+str(old_entropy - new_entropy))
  return old_entropy - new_entropy


In [ ]:
#test
from pprint import pprint
k = information_gain(data, 'Income', 'Attitude')
pprint(k)

Information Gain Calculation of  Income
target_attribute_name Buy_computer
Name:  high
Group:    Income Student Sex Buy_computer
2   high     yes   f          yes
3   high     yes   m          yes
Name:  low
Group:    Income Student Sex Buy_computer
4    low     yes   m           no
5    low     yes   f           no
7    low      no   f           no
Name:  medium
Group:     Income Student Sex Buy_computer
0  medium     yes   m          yes
1  medium      no   f           no
6  medium     yes   f          yes
8  medium      no   m           no
NOBS 9.0
-[1.0] * log2([1.0]) = sum([-0.0]) = 0.0
-[1.0] * log2([1.0]) = sum([-0.0]) = 0.0
-[0.5, 0.5] * log2([0.5, 0.5]) = sum([0.5, 0.5]) = 1.0
df_agg_ent         entropy_of_list  <lambda_0>
Income                             
high                0.0    0.222222
low                 0.0    0.333333
medium              1.0    0.444444
-[0.4444444444444444, 0.5555555555555556] * log2([0.4444444444444444, 0.5555555555555556]) = sum([0.51996666730769

#ID3 Algorithm 

In [ ]:
#ID3  Algorithm Function
def id3(df, target_attribute_name, attribute_names, default_class=None):
#Counting Total number of yes and no classes (Positive and negative Ex)
  from collections import Counter
  cnt = Counter(x for x in df[target_attribute_name])
  if len(cnt) == 1:
    return next(iter(cnt))
# Return None for Empty Data Set 
  elif df.empty or (not attribute_names):
    return default_class
  else:
    default_class = max(cnt.keys())
    print("attribute_names:",attribute_names)
    gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] 
    #Separating the maximum information gain attribute after calculating the information gain 
    index_of_max = gainz.index(max(gainz)) #Index of Best Attribute 
    best_attr = attribute_names[index_of_max] #choosing best attribute 
    #The tree is initially an empty dictionary
    tree = {best_attr:{}} # Initiate the tree with best attribute as a node 
    remaining_attribute_names = [i for i in attribute_names if i != best_attr]
    for attr_val, data_subset in df.groupby(best_attr):
      subtree = id3(data_subset,
                          target_attribute_name,
                          remaining_attribute_names,
                          default_class)
      tree[best_attr][attr_val] = subtree
  return tree

In [ ]:
attribute_names = list(data.columns)
print("List of Attributes:", attribute_names) 
attribute_names.remove('Attitude')
#Remove the class attribute 
print("Predicting Attributes:", attribute_names)

List of Attributes: ['Size', 'Color', 'Feel', 'Temperature', 'Attitude']
Predicting Attributes: ['Size', 'Color', 'Feel', 'Temperature']


In [ ]:
from pprint import pprint
tree = id3(data,'Attitude',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

attribute_names: ['Size', 'Color', 'Feel', 'Temperature']
Information Gain Calculation of  Size
target_attribute_name Attitude
Name:  big
Group:    Size   Color  Feel Temperature  Attitude
0  big  yellow  hard         low  positive
1  big  yellow  soft        high  negative
Name:  medium
Group:       Size   Color  Feel Temperature  Attitude
2  medium  yellow  soft        high  positive
3  medium    blue  hard        high  negative
4  medium    blue  soft         low  positive
5  medium    blue  soft         low  positive
NOBS 6.0
-[0.5, 0.5] * log2([0.5, 0.5]) = sum([0.5, 0.5]) = 1.0
-[0.75, 0.25] * log2([0.75, 0.25]) = sum([0.31127812445913283, 0.5]) = 0.8112781244591328
df_agg_ent         entropy_of_list  <lambda_0>
Size                               
big            1.000000    0.333333
medium         0.811278    0.666667
-[0.6666666666666666, 0.3333333333333333] * log2([0.6666666666666666, 0.3333333333333333]) = sum([0.38997500048077083, 0.5283208335737187]) = 0.9182958340544896
Ent

#ACCURACY 

In [ ]:
#Defining a function to calculate accuracy
def classify(instance, tree, default=None):
  attribute = next(iter(tree))
  print("Key:",tree.keys())
  print("Attribute:",attribute)
  print("Insance of Attribute :",instance[attribute],attribute)
  if instance[attribute] in tree[attribute].keys(): 
      result = tree[attribute][instance[attribute]]
      print("Instance Attribute:",instance[attribute],"TreeKeys :",tree[attribute].keys())
      if isinstance(result, dict): 
          return classify(instance, result)
      else:
          return result 
  else:
      return default

In [ ]:
training_data = data.iloc[1:-4] 

test_data  = data.iloc[-4:]

train_tree = id3(training_data, 'Buy_computer', attribute_names)

test_data['predicted'] = test_data.apply(  

classify, axis=1, args=(train_tree,'Yes') ) 

attribute_names: ['Income', 'Student', 'Sex']
Information Gain Calculation of  Income
target_attribute_name Buy_computer
Name:  high
Group:    Income Student Sex Buy_computer
2   high     yes   f          yes
3   high     yes   m          yes
Name:  low
Group:    Income Student Sex Buy_computer
4    low     yes   m           no
Name:  medium
Group:     Income Student Sex Buy_computer
1  medium      no   f           no
NOBS 4.0
-[1.0] * log2([1.0]) = sum([-0.0]) = 0.0
-[1.0] * log2([1.0]) = sum([-0.0]) = 0.0
-[1.0] * log2([1.0]) = sum([-0.0]) = 0.0
df_agg_ent         entropy_of_list  <lambda_0>
Income                             
high                0.0        0.50
low                 0.0        0.25
medium              0.0        0.25
-[0.5, 0.5] * log2([0.5, 0.5]) = sum([0.5, 0.5]) = 1.0
Entropy of COMP_07 Data(1.0) - 0.0
Info-gain is :1.0
Information Gain Calculation of  Student
target_attribute_name Buy_computer
Name:  no
Group:     Income Student Sex Buy_computer
1  medium      no 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
training_data['predicted'] = training_data.apply(classify, axis=1, args=(tree,'No') ) 

print(training_data['predicted'])

print('\n Accuracy is:\n' + str( sum(training_data['Buy_computer']==training_data['predicted'] ) / (1.0*len(training_data.index)) ))

training_data[['Buy_computer', 'predicted']]

Key: dict_keys(['Income'])
Attribute: Income
Insance of Attribute : medium Income
Instance Attribute: medium TreeKeys : dict_keys(['high', 'low', 'medium'])
Key: dict_keys(['Student'])
Attribute: Student
Insance of Attribute : no Student
Instance Attribute: no TreeKeys : dict_keys(['no', 'yes'])
Key: dict_keys(['Income'])
Attribute: Income
Insance of Attribute : high Income
Instance Attribute: high TreeKeys : dict_keys(['high', 'low', 'medium'])
Key: dict_keys(['Income'])
Attribute: Income
Insance of Attribute : high Income
Instance Attribute: high TreeKeys : dict_keys(['high', 'low', 'medium'])
Key: dict_keys(['Income'])
Attribute: Income
Insance of Attribute : low Income
Instance Attribute: low TreeKeys : dict_keys(['high', 'low', 'medium'])
1     no
2    yes
3    yes
4     no
Name: predicted, dtype: object

 Accuracy is:
1.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Buy_computer,predicted
1,no,no
2,yes,yes
3,yes,yes
4,no,no


In [ ]:
print ('\n\n Accuracy is : ' + str( sum(test_data['Buy_computer']==test_data['predicted'] ) / (1.0*len(test_data.index)) ))

test_data[['Buy_computer', 'predicted']]



 Accuracy is : 0.75


,Buy_computer,predicted
5,no,no
6,yes,no
7,no,no
8,no,no
